In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2
import os
import glob
import numpy as np
from tqdm.notebook import tqdm

BATCH_SIZE = 8
LEARNING_RATE = 1e-4
NUM_EPOCHS = 15
NUM_CLASSES = 3
SEQ_LENGTH = 16
LATENT_DIM = 512
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"🔥 Training on: {DEVICE}")


POSSIBLE_PATHS = [
    '/kaggle/input/boxing-processed-finaldata',
    '/kaggle/input/boxing-processed-finaldata/processed_dataset',
    '/kaggle/input/boxing-final-processed-v1/processed_dataset'
]

DATA_ROOT = None

for path in POSSIBLE_PATHS:
    if os.path.exists(path) and os.path.isdir(path):
        if os.path.exists(os.path.join(path, 'Jab')):
            DATA_ROOT = path
            break


if DATA_ROOT is None:
    root_input = '/kaggle/input'
    for d in os.listdir(root_input):
        full_path = os.path.join(root_input, d)
        if os.path.isdir(full_path) and 'Jab' in os.listdir(full_path):
            DATA_ROOT = full_path
            break

if DATA_ROOT:
    print(f"Data Source Found: {DATA_ROOT}")
else:
    raise FileNotFoundError("Error: Could not find the dataset path! Please check 'Add Input'.")


class BoxingVideoDataset(Dataset):
    def __init__(self, root_dir, sequence_length=16, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.sequence_length = sequence_length
        self.classes = ['Jab', 'Cross', 'Hook']
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        self.videos = []
        self.labels = []
        
        for cls in self.classes:
            cls_path = os.path.join(root_dir, cls)
            if not os.path.exists(cls_path): continue
            
            files = glob.glob(f"{cls_path}/*.mp4") + glob.glob(f"{cls_path}/*.MP4")
            self.videos.extend(files)
            self.labels.extend([self.class_to_idx[cls]] * len(files))
            
    def __len__(self):
        return len(self.videos)
    
    def __getitem__(self, idx):
        video_path = self.videos[idx]
        label = self.labels[idx]
        
        cap = cv2.VideoCapture(video_path)
        frames = []
        while len(frames) < self.sequence_length:
            ret, frame = cap.read()
            if not ret: break
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            if self.transform:
                frame = self.transform(frame)
            frames.append(frame)
        cap.release()
        
        # Padding if video is too short
        if len(frames) == 0: 
            return torch.zeros(3, self.sequence_length, 224, 224), label
            
        while len(frames) < self.sequence_length:
            frames.append(frames[-1])
            
        video_tensor = torch.stack(frames)
        if self.transform:
             video_tensor = video_tensor.permute(1, 0, 2, 3)
             
        return video_tensor, label


train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Loading Dataset into Memory...")
dataset = BoxingVideoDataset(DATA_ROOT, transform=train_transforms)
train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
print(f"Loaded {len(dataset)} videos.")


class CustomCNN(nn.Module):
    def __init__(self, out_dim=512):
        super(CustomCNN, self).__init__()
        # 4 Blocks of Conv -> BN -> ReLU -> MaxPool
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2)
        )
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256, out_dim)
        
    def forward(self, x):
        x = self.features(x)
        x = self.avg_pool(x).flatten(1)
        return self.fc(x)

class HandCodedSelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(HandCodedSelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads
        
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        energy = torch.einsum("nqhd,nkhd->nhqk", [self.queries(queries), self.keys(keys)])
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        out = torch.einsum("nhql,nlhd->nqhd", [attention, self.values(values)]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        return self.fc_out(out)

class ActionNet(nn.Module):
    def __init__(self, num_classes=3, latent_dim=512, num_heads=8):
        super(ActionNet, self).__init__()
        self.backbone = CustomCNN(out_dim=latent_dim)
        self.attention = HandCodedSelfAttention(embed_size=latent_dim, heads=num_heads)
        encoder_layer = nn.TransformerEncoderLayer(d_model=latent_dim, nhead=num_heads, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.classifier = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(latent_dim, 256), nn.ReLU(), nn.Linear(256, num_classes)
        )

    def forward(self, x):
        batch_size, C, frames, H, W = x.shape
        c_in = x.view(batch_size * frames, C, H, W)
        features = self.backbone(c_in).view(batch_size, frames, -1)
        attn_out = self.attention(features, features, features, mask=None)
        transformer_out = self.transformer_encoder(attn_out + features)
        return self.classifier(torch.mean(transformer_out, dim=1))


model = ActionNet(num_classes=NUM_CLASSES, latent_dim=LATENT_DIM).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("STARTING TRAINING (ActionNet - Custom Backbone)...")
print("-" * 30)

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    
    loop = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", leave=True)
    
    for videos, labels in loop:
        videos, labels = videos.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = model(videos)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * videos.size(0)
        _, predicted = torch.max(outputs, 1)
        total_preds += labels.size(0)
        correct_preds += (predicted == labels).sum().item()
        
        loop.set_postfix(loss=loss.item())
        
    epoch_loss = running_loss / len(dataset)
    epoch_acc = correct_preds / total_preds
    print(f"Epoch {epoch+1} -> Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc*100:.2f}%")

print("="*30)
torch.save(model.state_dict(), "ActionNet_Custom_Final.pth")
print("Model Saved Successfully!")

🔥 Training on: cuda
Data Source Found: /kaggle/input/boxing-processed-finaldata
Loading Dataset into Memory...
Loaded 605 videos.
STARTING TRAINING (ActionNet - Custom Backbone)...
------------------------------


Epoch 1/15:   0%|          | 0/76 [00:00<?, ?it/s]

Epoch 1 -> Loss: 0.6072 | Accuracy: 74.21%


Epoch 2/15:   0%|          | 0/76 [00:00<?, ?it/s]

Epoch 2 -> Loss: 0.4977 | Accuracy: 77.69%


...

Epoch 13 -> Loss: 0.4929 | Accuracy: 78.02%


Epoch 14/15:   0%|          | 0/76 [00:00<?, ?it/s]

Epoch 14 -> Loss: 0.4897 | Accuracy: 78.02%


Epoch 15/15:   0%|          | 0/76 [00:00<?, ?it/s]

Epoch 15 -> Loss: 0.5002 | Accuracy: 78.02%
Model Saved Successfully!


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2
import os
import glob
import numpy as np
from tqdm.notebook import tqdm

BATCH_SIZE = 16          
LEARNING_RATE = 1e-4
NUM_EPOCHS = 10         
NUM_CLASSES = 2          # Land, Miss
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Training HitNet on: {DEVICE}")


if 'DATA_ROOT' not in globals():
    POSSIBLE_PATHS = [
        '/kaggle/input/boxing-processed-finaldata',
        '/kaggle/input/boxing-processed-finaldata/processed_dataset',
        '/kaggle/input/boxing-final-processed-v1/processed_dataset'
    ]
    DATA_ROOT = None
    for path in POSSIBLE_PATHS:
        if os.path.exists(path) and os.path.isdir(path):
            if os.path.exists(os.path.join(path, 'punch_imgs')): # بندور على فولدر الصور
                DATA_ROOT = path
                break
    
    if DATA_ROOT is None:
      
        root_input = '/kaggle/input'
        for d in os.listdir(root_input):
            full_path = os.path.join(root_input, d)
            if os.path.isdir(full_path) and 'punch_imgs' in os.listdir(full_path):
                DATA_ROOT = full_path
                break

if DATA_ROOT:
    IMAGES_ROOT = os.path.join(DATA_ROOT, 'punch_imgs')
    print(f"Images Found at: {IMAGES_ROOT}")
else:
    raise FileNotFoundError("Error: Could not find 'punch_imgs' folder!")

class BoxingImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['Miss', 'Land'] # الترتيب مهم: Miss=0, Land=1
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        self.images = []
        self.labels = []
        
        for cls in self.classes:
            cls_path = os.path.join(root_dir, cls)
            if not os.path.exists(cls_path): continue
            
            files = glob.glob(f"{cls_path}/*.jpg") + glob.glob(f"{cls_path}/*.png")
            self.images.extend(files)
            self.labels.extend([self.class_to_idx[cls]] * len(files))
            
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

img_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), # Data Augmentation بسيط
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Loading HitNet Dataset...")
hit_dataset = BoxingImageDataset(IMAGES_ROOT, transform=img_transforms)
hit_loader = DataLoader(hit_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
print(f"Loaded {len(hit_dataset)} images for HitNet.")

class HandCodedSelfAttention(nn.Module):
    def __init__(self, embed_size, heads):
        super(HandCodedSelfAttention, self).__init__()
        self.embed_size = embed_size
        self.heads = heads
        self.head_dim = embed_size // heads
        
        self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
        self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

    def forward(self, values, keys, query, mask):
        N = query.shape[0]
        value_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

        values = values.reshape(N, value_len, self.heads, self.head_dim)
        keys = keys.reshape(N, key_len, self.heads, self.head_dim)
        queries = query.reshape(N, query_len, self.heads, self.head_dim)

        energy = torch.einsum("nqhd,nkhd->nhqk", [self.queries(queries), self.keys(keys)])
        attention = torch.softmax(energy / (self.embed_size ** (1 / 2)), dim=3)
        out = torch.einsum("nhql,nlhd->nqhd", [attention, self.values(values)]).reshape(
            N, query_len, self.heads * self.head_dim
        )
        return self.fc_out(out)

class HitNet(nn.Module):
    def __init__(self):
        super(HitNet, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.BatchNorm2d(32), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.MaxPool2d(2)
        ) 
        
        self.attention = HandCodedSelfAttention(embed_size=256, heads=4)
        
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 2) # 2 classes: Miss, Land
        )

    def forward(self, x):
        batch_size = x.shape[0]
        
        x = self.features(x) # (B, 256, 14, 14)
        
        x = x.permute(0, 2, 3, 1).view(batch_size, -1, 256) # (B, 196, 256)
        
        attn_out = self.attention(x, x, x, mask=None)
        
        x = torch.mean(attn_out, dim=1) # (B, 256)
        
        out = self.classifier(x)
        return out


hit_model = HitNet().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(hit_model.parameters(), lr=LEARNING_RATE)

print("STARTING HITNET TRAINING...")
print("-" * 30)

for epoch in range(NUM_EPOCHS):
    hit_model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    
    loop = tqdm(hit_loader, desc=f"HitNet Epoch {epoch+1}/{NUM_EPOCHS}", leave=True)
    
    for images, labels in loop:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        
        optimizer.zero_grad()
        outputs = hit_model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        total_preds += labels.size(0)
        correct_preds += (predicted == labels).sum().item()
        
        loop.set_postfix(loss=loss.item())
        
    epoch_loss = running_loss / len(hit_dataset)
    epoch_acc = correct_preds / total_preds
    print(f"Epoch {epoch+1} -> Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc*100:.2f}%")

print("="*30)
torch.save(hit_model.state_dict(), "HitNet_Final.pth")
print("HitNet Saved Successfully: HitNet_Final.pth")

Training HitNet on: cuda
Images Found at: /kaggle/input/boxing-processed-finaldata/punch_imgs
Loading HitNet Dataset...
Loaded 744 images for HitNet.
STARTING HITNET TRAINING...
------------------------------


HitNet Epoch 1/10:   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 1 -> Loss: 0.5107 | Accuracy: 80.65%


HitNet Epoch 2/10:   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 2 -> Loss: 0.4677 | Accuracy: 81.32%


...

Epoch 8 -> Loss: 0.4558 | Accuracy: 81.32%


HitNet Epoch 9/10:   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 9 -> Loss: 0.4510 | Accuracy: 81.32%


HitNet Epoch 10/10:   0%|          | 0/47 [00:00<?, ?it/s]

Epoch 10 -> Loss: 0.4556 | Accuracy: 81.32%
HitNet Saved Successfully: HitNet_Final.pth


In [3]:
import os
from IPython.display import FileLink


print("Checking for saved models in working directory...")
files = os.listdir('/kaggle/working')

model_files = [f for f in files if f.endswith('.pth')]

if len(model_files) > 0:
    print(f"Found {len(model_files)} models: {model_files}")
    print("\nCLICK BELOW TO DOWNLOAD (Important!):")
    print("-" * 30)
    
   
    for model_file in model_files:
        display(FileLink(model_file))
else:
    print("No .pth files found! Did the training finish?")

Checking for saved models in working directory...
Found 2 models: ['HitNet_Final.pth', 'ActionNet_Custom_Final.pth']

CLICK BELOW TO DOWNLOAD (Important!):
------------------------------


/kaggle/working/HitNet_Final.pth

/kaggle/working/ActionNet_Custom_Final.pth